In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import builtwith
import mysql.connector as my
import time

In [6]:
data=pd.read_csv("D:/Bigcommerce UnitedKindgom.csv")
domains=data['Domain']
# domains=domains[22505:]
domains

0            ordnancesurvey.co.uk
1           square-enix-games.com
2            tottenhamhotspur.com
3                 raspberrypi.com
4                    napier.ac.uk
                  ...            
2408               tsstools.co.uk
2409    tonys-trading-cards.co.uk
2410                 topken.co.uk
2411             torqeedoshop.com
2412         toydogsoldiers.co.uk
Name: Domain, Length: 2413, dtype: object

In [14]:
domains[domains=='danlexgift.com']

44526    danlexgift.com
Name: Domain, dtype: object

In [5]:
domains= data[~data['Domain'].isin(df['Domain'])]
domains=domains['Domain']
domains

383075            ipetsfy.com
383076       ipetshoppers.com
383077            ipetsie.com
383078             ipetso.com
383079           ipetsore.com
                  ...        
3288578      tonyshobbies.com
3288579         tonyshoes.com
3288580      tonytimes.com.co
3288581    tonyviramontes.com
3288582     tonywinstudio.com
Name: Domain, Length: 2905508, dtype: object

In [240]:
domains.duplicated().sum()

0

In [3]:
con= my.connect(host='localhost',user='root',password='Jasper@1998', database= 'Techstack')
cursor=con.cursor()
cursor.execute('SHOW tables') #executing show database to check the databases present in MYSQL server
cursor.fetchall() 

[('automotive',),
 ('aws',),
 ('cms_tech',),
 ('magento_1',),
 ('payment_techstack',),
 ('shopify',)]

In [ ]:
cursor.execute('Select * from shopify')
check=cursor.fetchall()
col= [colo[0] for colo in cursor.description]
df= pd.DataFrame(check , columns=col)
df

In [8]:
table_query=''' CREATE TABLE Bigcommerce (
                Domain varchar(255),
                Ecommerce_last varchar(255),
                Ecommerce_first varchar(255))'''
cursor.execute(table_query)
cursor.execute('SHOW tables') 
cursor.fetchall() 

[('automotive',),
 ('aws',),
 ('bigcommerce',),
 ('cms_tech',),
 ('magento_1',),
 ('payment_techstack',),
 ('shopify',)]

In [10]:
cursor.execute('Alter table bigcommerce add column created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP')

In [12]:
insert_query = "INSERT INTO bigcommerce (Domain, list_name) VALUES (%s,%s)"
# update_query = "UPDATE bigcommerce SET Ecommerce_last = %s, Version_last = %s, Ecommerce_first = %s, Version_first = %s, list_name= %s WHERE URL = %s"
update_query = "UPDATE bigcommerce SET Ecommerce_last = %s,  Ecommerce_first = %s WHERE Domain = %s"
list_name = "Bigcommerce_UK"
for domain in domains:
    url = "https://builtwith.com/"+ domain
    
    # Retry logic
    retries = 3
    while retries > 0:
        try:
            response = requests.get(url)
            break  # Break out of the loop if request succeeds
        except requests.exceptions.ConnectionError as e:
            print(f"Connection error occurred: {e}. Retrying...")
            retries -= 1
            time.sleep(5)  # Wait for 5 seconds before retrying

    soup = BeautifulSoup(response.content, 'html.parser')
    div_tags = soup.find_all('div', class_='card mt-4 mb-2')

    # Insert domain into the database
    cursor.execute(insert_query, (domain,list_name))
    con.commit()

    for div_tag in div_tags:
        if div_tag and 'eCommerce' in div_tag.find('h6', class_='card-title'):
            tags = div_tag.find_all('div', class_='row mb-1 mt-1')
            try:
                ecommerce_last = tags[-1].find('h2').text
                ecommerce_first = tags[0].find('h2').text
            except:
                ecommerce_last = 'Not mentioned'
                ecommerce_first = 'Not mentioned'

            try:
                version2 = tags[-1].find('h5').text
            except:
                version2 = 'NA'

            try:
                Version1 = tags[0].find('h5').text
            except:
                Version1 = 'NA'

#             Update the database with retrieved values
            cursor.execute(update_query, (ecommerce_last, ecommerce_first, domain))
#             cursor.execute(update_query, (ecommerce_last, version2, ecommerce_first,Version1,list_name, domain))
            con.commit()


Connection error occurred: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')). Retrying...


In [15]:
query ='''SELECT * FROM Bigcommerce;'''

cursor.execute(query)
check=cursor.fetchall()
col= [colo[0] for colo in cursor.description]
cart_df= pd.DataFrame(check , columns=col)
cart_df

,Domain,Ecommerce_last,Ecommerce_first,list_name,created_at,Techstack
0,ordnancesurvey.co.uk,None,None,Bigcommerce_UK,2024-04-17 16:54:14,Other Ecommerce
1,square-enix-games.com,None,None,Bigcommerce_UK,2024-04-17 16:54:15,Other Ecommerce
2,tottenhamhotspur.com,Cart Functionality,Cart Functionality,Bigcommerce_UK,2024-04-17 16:54:16,Other Ecommerce
3,raspberrypi.com,BigCommerce,BigCommerce,Bigcommerce_UK,2024-04-17 16:54:16,Bigcommerce
4,napier.ac.uk,None,None,Bigcommerce_UK,2024-04-17 16:54:17,Other Ecommerce
...,...,...,...,...,...,...
2408,tsstools.co.uk,BigCommerce,BigCommerce,Bigcommerce_UK,2024-04-17 17:22:52,Bigcommerce
2409,tonys-trading-cards.co.uk,BigCommerce,BigCommerce,Bigcommerce_UK,2024-04-17 17:22:53,Bigcommerce
2410,topken.co.uk,BigCommerce,BigCommerce,Bigcommerce_UK,2024-04-17 17:22:53,Bigcommerce
2411,torqeedoshop.com,BigCommerce,BigCommerce,Bigcommerce_UK,2024-04-17 17:22:54,Bigcommerce


In [39]:
tech_datas=pd.DataFrame(tech_data, columns=['Domain', 'Ecommerce_last', 'Version_last', 'Ecommerce_first', 'Version_first'])

In [16]:
final_output= pd.merge(data, cart_df, on='Domain', how='inner')
final_output

,Domain,Location on Site,Vertical,Employees,Tranco,Page Rank,Majestic,Umbrella,LinkedIn,City,State,Country,Last Indexed,Ecommerce_last,Ecommerce_first,list_name,created_at,Techstack
0,ordnancesurvey.co.uk,shop.ordnancesurvey.co.uk,Business And Industrial,0.0,103723,6008,10468,209767,linkedin.com/company/ordnance-survey,Nursling,ENG,GB,21-02-2024,None,None,Bigcommerce_UK,2024-04-17 16:54:14,Other Ecommerce
1,square-enix-games.com,na.store.square-enix-games.com;au.store.square...,Art And Entertainment,0.0,467424,10705,9594,30376,NaN,London Bridge,ENG,GB,21-02-2024,None,None,Bigcommerce_UK,2024-04-17 16:54:15,Other Ecommerce
2,tottenhamhotspur.com,shop.tottenhamhotspur.com,Business And Industrial,0.0,22608,11094,11270,127729,NaN,Waltham Forest,ENG,GB,17-02-2024,Cart Functionality,Cart Functionality,Bigcommerce_UK,2024-04-17 16:54:16,Other Ecommerce
3,raspberrypi.com,raspberrypi.com,Art And Entertainment,10.0,338084,15873,252512,Outside Top 1m,linkedin.com/company/raspberrypi,Cambridge,CAM,GB,17-02-2024,BigCommerce,BigCommerce,Bigcommerce_UK,2024-04-17 16:54:16,Bigcommerce
4,napier.ac.uk,my.napier.ac.uk;onlineshop.napier.ac.uk,Education,1000.0,136725,18138,13076,196031,NaN,Edinburgh,SCT,GB,21-02-2024,None,None,Bigcommerce_UK,2024-04-17 16:54:17,Other Ecommerce
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2408,tsstools.co.uk,tsstools.co.uk,NaN,0.0,Outside Top 1m,100000000,Outside Top 1m,Outside Top 1m,NaN,London,ENG,GB,10-02-2024,BigCommerce,BigCommerce,Bigcommerce_UK,2024-04-17 17:22:52,Bigcommerce
2409,tonys-trading-cards.co.uk,tonys-trading-cards.co.uk,Finance,NaN,Outside Top 1m,100000000,Outside Top 1m,Outside Top 1m,NaN,Aberdeen City,UK,GB,10-02-2024,BigCommerce,BigCommerce,Bigcommerce_UK,2024-04-17 17:22:53,Bigcommerce
2410,topken.co.uk,topken.co.uk,Law Govt And Politics,0.0,Outside Top 1m,100000000,Outside Top 1m,Outside Top 1m,NaN,Luton,ENG,GB,11-02-2024,BigCommerce,BigCommerce,Bigcommerce_UK,2024-04-17 17:22:53,Bigcommerce
2411,torqeedoshop.com,torqeedoshop.com,Travel,0.0,Outside Top 1m,100000000,Outside Top 1m,Outside Top 1m,NaN,NaN,SCT,GB,15-02-2024,BigCommerce,BigCommerce,Bigcommerce_UK,2024-04-17 17:22:54,Bigcommerce


In [19]:
final_output.drop(columns=['Ecommerce_last','Ecommerce_first', 'list_name']).to_csv('D:/Bigcommerce UK Data Final List with techstack.csv', index=False)

In [50]:
final_output['Domain'].duplicated().sum()

7611

In [52]:
final_output.drop_duplicates(subset=['Domain'],inplace=True)

In [232]:
final_output.groupby('Techstack', dropna=False).size()

Techstack
Magento 1                 14679
Magento 2                 21686
Magento Enterprise          554
No Techstack available     5700
Other Ecommerce            2529
dtype: int64

In [235]:
final_output.to_csv("C:/Users/innoppl/Techstack/2024 Techstacks/Magento_2024 Final Techstack updated.csv")